In [ ]:
import Evaluasi
import numpy as np
import matplotlib.pyplot as plt 
import h5py
import cv2
import pickle
import tensorflow as tf
from tensorflow.keras.models import load_model

loss = []
val_loss = []
with open('Result/History_Train.pkl','rb') as f:
    log = pickle.load(f)
    for i in range(len(log)):
        loss.append(np.amin(log[i].get('loss')))
        val_loss.append(np.amin(log[i].get('val_loss')))

indeks = np.where(val_loss == np.amin(val_loss))[0]
predModel = load_model('Result/Fold'+str(indeks[0]+1)+'.hdf5')

In [ ]:
def preprocessing(ori):
    h,w,c = ori.shape
    crop_img = ori[120:120+h, 960:960+w]
    res = cv2.resize(crop_img,dsize=(320,320),interpolation=cv2.INTER_CUBIC)
    gray = cv2.cvtColor(res,cv2.COLOR_BGR2GRAY)
    return crop_img,gray

cap = cv2.VideoCapture('Lokasi video')
rasio = (960/320)

if (cap.isOpened()==False):
    print("Error")

while(cap.isOpened()):
    ret,frame = cap.read()
    if ret == True:
        show,prep = preprocessing(frame)
        heatmapPrediction = predModel.predict(np.expand_dims(prep,axis=[0,-1]))
        
        mean = np.mean([heatmapPrediction[0,:,:,i].max() for i in range(heatmapPrediction.shape[-1])])
        print(mean)
        
        keypointPred = Evaluasi.KeypointPrediction(heatmapPrediction[0])
        if len(keypointPred)>0:
            keypointPred = keypointPred*rasio
            # print(keypointPred)
            color = (0, 255, 0)
            cv2.imshow('Heatmap',np.sum(np.squeeze(heatmapPrediction,axis=0),axis=-1))

            if mean>0.3:
                for j in range(keypointPred.shape[0]):
                    cv2.polylines(show,[np.array([keypointPred[j,k,l,::-1]for l in range(keypointPred.shape[2])for k in range(keypointPred.shape[1])],np.int32)], True, color, 2)

                    # box = keypointPred[j,0,:,:]
                    # start = (int(box[0,1]),int(box[0,0]))
                    # end = (int(box[1,1]),int(box[1,0]))
                    # cv2.rectangle(show,start,end,color,3)

            cv2.imshow('Frame',show)

            key = cv2.waitKey(1)
            if key == ord('q'):
                break
            if key == ord('p'):
                cv2.waitKey(-1)
    else:
        break

cap.release()
cv2.destroyAllWindows()